In [25]:
import argparse
import time
import yaml
import os
import logging
import numpy as np
from collections import OrderedDict
from contextlib import suppress
from datetime import datetime
from spikingjelly.clock_driven import functional
from spikingjelly.datasets.cifar10_dvs import CIFAR10DVS
from spikingjelly.datasets.n_caltech101 import NCaltech101
import torch
import torch.nn as nn
import torchvision.utils
from torch.nn.parallel import DistributedDataParallel as NativeDDP
import torchinfo
import random as rd
from torchviz import make_dot
from torch.profiler import profile, record_function, ProfilerActivity
import csv

from timm.data import (
    create_dataset,
    create_loader,
    resolve_data_config,
    Mixup,
    FastCollateMixup,
    AugMixDataset,
)
from timm.models import (
    create_model,
    safe_model_name,
    resume_checkpoint,
    load_checkpoint,
    convert_splitbn_model,
    model_parameters,
)
from timm.models.helpers import clean_state_dict
from timm.utils import *
from timm.loss import (
    LabelSmoothingCrossEntropy,
    SoftTargetCrossEntropy,
    JsdCrossEntropy,
    BinaryCrossEntropy,
)
from timm.optim import create_optimizer_v2, optimizer_kwargs
from timm.scheduler import create_scheduler
from timm.utils import ApexScaler, NativeScaler
import model, dvs_utils

Command example to run the test.py

```python test.py -c conf/imagenet/8_768_200E_t4.yml --model sdt --spike-mode lif --attention_mode STAtten```

In [26]:
args_dict = {
    'model': 'sdt', # Name of model (default: "countception")
    'time_steps': 4,
    'pretrained': False,
    'drop': 0.0,
    'drop_path': 0.2,
    'drop_block': None,
    'num_heads': 8,
    'num_classes': 1000,
    'pooling_stat': '1111', # pooling layers in embedding layer
    'img_size': 32,
    'dim': 768,
    'mlp_ratio': 4,
    'in_channels': 3,
    'layer': 8,
    'patch_size': None, # config
    'spike_mode': 'lif', # config
    'dvs_mode': False,
    'TET': False
}

In [27]:
model = create_model(
    model_name=args_dict['model'],
    T=args_dict['time_steps'],
    pretrained=args_dict['pretrained'],
    drop_rate=args_dict['drop'],
    drop_path_rate=args_dict['drop_path'],
    drop_block_rate=args_dict['drop_block'],
    num_heads=args_dict['num_heads'],
    num_classes=args_dict['num_classes'],
    pooling_stat=args_dict['pooling_stat'],
    img_size_h=args_dict['img_size'],
    img_size_w=args_dict['img_size'],
    patch_size=args_dict['patch_size'],
    embed_dims=args_dict['dim'],
    mlp_ratios=args_dict['mlp_ratio'],
    in_channels=args_dict['in_channels'],
    qkv_bias=False,
    depths=args_dict['layer'],
    sr_ratios=1,
    spike_mode=args_dict['spike_mode'],
    dvs_mode=args_dict['dvs_mode'],
    TET=args_dict['TET'],
)

In [28]:
print(model)

SpikeDrivenTransformer(
  (patch_embed): MS_SPS(
    (proj_conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (proj_bn): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (proj_lif): MultiStepLIFNode(
      v_threshold=1.0, v_reset=0.0, detach_reset=True, tau=2.0, backend=cupy
      (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
    )
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (proj_conv1): Conv2d(96, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (proj_bn1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (proj_lif1): MultiStepLIFNode(
      v_threshold=1.0, v_reset=0.0, detach_reset=True, tau=2.0, backend=cupy
      (surrogate_function): Sigmoid(alpha=4.0, spiking=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (proj_conv2): Conv2d(192, 

In [29]:
test_tensor =torch.randn(1, 3, 32, 32)
test_tensor = test_tensor.to('cuda')
model = model.to('cuda')
output = model(test_tensor)[0]

[Time] - SPS.PSM: 0.0020 seconds
[Time] - SPS.RPE: 0.0006 seconds
sparsity x_for_qkv: 0.7255045572916667
shape x_for_qkv: torch.Size([4, 768, 2, 2])
sparsity q: 0.9737141927083334
q shape: torch.Size([4, 1, 8, 4, 96])
sparsity k: 0.971923828125
k shape: torch.Size([4, 1, 8, 4, 96])
sparsity v: 0.9707845052083334
v shape: torch.Size([4, 1, 8, 4, 96])
sparsity Attention q_chunks: 0.9737141927083334
sparsity Attention k_chunks: 0.971923828125
sparsity Attention v_chunks: 0.9707845052083334
[Time] - Reshape: 0.0003 seconds
sparsity Attention postReshape q_chunks: 0.9737141927083334
sparsity Attention postReshape k_chunks: 0.971923828125
sparsity Attention postReshape v_chunks: 0.9707845052083334
q_chunks shape: torch.Size([2, 1, 8, 8, 96])
k_chunks shape: torch.Size([2, 1, 8, 8, 96])
v_chunks shape: torch.Size([2, 1, 8, 8, 96])
[Time] - Matmul: 0.0002 seconds
sparsity Attention out: 0.9756673177083334
[Time] - Reshape Time-Space: 0.0000 seconds
shape : torch.Size([4, 1, 8, 4, 96])
sparsity

In [30]:
import torchvision
dataset_cifar=torchvision.datasets.CIFAR10("./cifar10", train=False, download=True, transform=torchvision.transforms.ToTensor())
dataset_cifar100=torchvision.datasets.CIFAR100("./cifar100", train=False, download=True, transform=torchvision.transforms.ToTensor())
dataset_cifar100img=torchvision.datasets.CIFAR100("./cifar100", train=False, download=True)


In [31]:
tensor = dataset_cifar100.__getitem__(8)[1]
print(tensor)
test_tensor = dataset_cifar100.__getitem__(8)[0]
test_tensor = test_tensor.unsqueeze(0)


23


In [ ]:
%%capture cap

# test_tensor =torch.randn(1, 3, 224, 224)
# test_tensor = test_tensor.to('cuda')
# model = model.to('cuda')
# output = model(test_tensor)[0]
# output.shape
# output

import io
import sys
import re
import pandas as pd

# Dizionario per raccogliere i tempi di ciascuna componente
tempi_componenti = {}

# Numero di esecuzioni
n_runs = 30

for i in range(n_runs):
    try:
        # test_tensor = dataset_cifar100.__getitem__(i)[0]
        # test_tensor = test_tensor.unsqueeze(0)
        # test_tensor =torch.randn(1, 3, 32, 32)
        test_tensor =torch.randn(1, 3, 32, 32)
        test_tensor = test_tensor.to('cuda')
        model = model.to('cuda')
        init_time_total_model_time = time.time()
        output = model(test_tensor)[0]
        # del test_tensor
        total_time_model = time.time() - init_time_total_model_time
        print(f"Run {i+1} - Tempo totale del modello: {total_time_model:.4f} secondi")
    except Exception as e:
        print(f"Errore durante l'esecuzione del modello: {e}")
        continue
    finally:
        with open('capcifarTensRandn.txt', 'w') as file:
            file.write(cap.stdout)



# #Profile
# with profile(
#     activities=[ProfilerActivity.CPU],  # aggiungi CUDA se serve
#     record_shapes=True,
#     profile_memory=True,
#     with_stack=True
# ) as prof:
#     with record_function("model_inference"):
#         model(input_tensor)
        
# # Salva i dati principali del profiling
# with open("profiling_output.csv", "w", newline="") as f:
#     writer = csv.writer(f)
#     writer.writerow(["name", "cpu_time_total (us)", "self_cpu_time_total (us)", "input_shapes"])
    
#     for evt in prof.key_averages():
#         writer.writerow([
#             evt.key,
#             evt.cpu_time_total,
#             evt.self_cpu_time_total,
#             str(evt.input_shapes)
#         ])


# Save the model graph
#output_path = "./images/"
#graph = make_dot(output, params=dict(model.named_parameters()))
#graph.render("SDT", format="png", cleanup=True).save(output_path)
